In [ ]:
!pip install pandas
!pip install pygeohash
!pip install confluent_kafka

In [ ]:
import pandas as pd

In [ ]:
taxis_df = pd.read_csv("taxis_porto.csv")

In [ ]:
ten_taxis_df = taxis_df[['TAXI_ID','POLYLINE']].copy()

In [ ]:
import time
import json

from confluent_kafka import Producer

host = 'endpoint:port,endpoint:port'

p = Producer({'bootstrap.servers': host})

def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))
        
def sendToKafka(data):

    p.poll(0)

    p.produce('AWSKafkaTutorialTopic', data.encode('utf-8'), callback=delivery_report)

    p.flush()

In [ ]:
import time
import json
import pygeohash as pgh

coordenada_index = 0
while coordenada_index < 100:
    print('coordenada ' + str(coordenada_index))

    for taxi_index, row in ten_taxis_df.iterrows():

        taxi_id = row['TAXI_ID']
        polyline = row['POLYLINE']
        polyline[0]

        # parse x:
        polyline_json = json.loads(polyline)

        if coordenada_index < len(polyline_json) :
            data_set = {'text': 't' + str(taxi_id), "lat": polyline_json[coordenada_index][1], "lon": polyline_json[coordenada_index][0]}

            json_dump = json.dumps(data_set)
            print(json_dump)
            sendToKafka(json_dump)
        else :
            print('')

        taxi_index += 1

    coordenada_index += 1
    time.sleep(5)